In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install barbar

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import random
import numpy as np
import os

from barbar import Bar
%matplotlib inline

In [ ]:
EPOCH = 200
C = 100
BATCH = 1024
PATH = '/content/drive/My Drive/visual_media/mixup'

writer = SummaryWriter(log_dir=PATH)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

SEED = 2020
seed_everything(SEED)

In [ ]:
def mixup(batch, alpha=1.0):
    input, target = batch

    ### Shuffle Minibatch ###
    indices = torch.randperm(input.size(0))
    input_s, target_s = input[indices], target[indices]

    lamb = np.random.beta(alpha, alpha)

    input = lamb * input + (1-lamb) * input_s
    target = (target, target_s, lamb)

    return input, target

In [ ]:
transform = transforms.Compose([
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH,
                                          shuffle=True, num_workers=2)

test_transform = transforms.Compose([
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=test_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
net = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=False).cuda()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)

In [ ]:
for epoch in range(EPOCH):
    
    ################
    ### Training ###
    ################
    running_loss = 0.0
    for i, data in enumerate(Bar(trainloader)):
        data = mixup(data)
        inputs, labels = data[0].cuda(), data[1]

        optimizer.zero_grad()

        outputs = net(inputs)

        target, target_s, lamb = labels[0].cuda(), labels[1].cuda(), labels[2]
        loss = lamb * criterion(outputs, target) + (1 - lamb) * criterion(outputs, target_s)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print('[%d] loss: %.3f' %
          (epoch + 1, running_loss / 50000 * i))
    writer.add_scalar("loss", running_loss / 50000 * i, epoch)
    running_loss = 0.0


    ################
    ### Testing  ###
    ################
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].cuda(), data[1].cuda()
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (
        100 * correct / total))
    writer.add_scalar("test_accuracy", 100 * correct / total, epoch)

    torch.save(net.state_dict(), PATH + '/model_latest.h5')
    scheduler.step()

50000/50000: [===============================>] - ETA 2.8s
[1] loss: 0.235
Accuracy of the network on the 10000 test images: 12 %
50000/50000: [===============================>] - ETA 0.6s
[2] loss: 0.189
Accuracy of the network on the 10000 test images: 18 %
50000/50000: [===============================>] - ETA 0.6s
[3] loss: 0.180
Accuracy of the network on the 10000 test images: 22 %
50000/50000: [===============================>] - ETA 0.6s
[4] loss: 0.174
Accuracy of the network on the 10000 test images: 24 %
50000/50000: [===============================>] - ETA 0.6s
[5] loss: 0.170
Accuracy of the network on the 10000 test images: 25 %
50000/50000: [===============================>] - ETA 0.6s
[6] loss: 0.165
Accuracy of the network on the 10000 test images: 27 %
50000/50000: [===============================>] - ETA 0.6s
[7] loss: 0.162
Accuracy of the network on the 10000 test images: 29 %
50000/50000: [===============================>] - ETA 0.6s
[8] loss: 0.159
Accuracy of the

In [ ]:
torch.save(net.state_dict(), PATH + '/model.h5')

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].cuda(), data[1].cuda()
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %.3f %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 41.670 %
